**Mining Process Flotation Plant Database**

The main goal is to use this data to predict how much impurity is in the ore concentrate. As this impurity is measured every hour, if we can predict how much silica (impurity) is in the ore concentrate, we can help the engineers, giving them early information to take actions (empowering!). Hence, they will be able to take corrective actions in advance (reduce impurity, if it is the case) and also help the environment (reducing the amount of ore that goes to tailings as you reduce silica in the ore concentrate).

More Information on the dataset: 

The first column shows time and date range (from march of 2017 until september of 2017). Some columns were sampled every 20 second. Others were sampled on a hourly base.

The second and third columns are quality measures of the iron ore pulp right before it is fed into the flotation plant. Column 4 until column 8 are the most important variables that impact in the ore quality in the end of the process. From column 9 until column 22, we can see process data (level and air flow inside the flotation columns, which also impact in ore quality. The last two columns are the final iron ore pulp quality measurement from the lab.
Target is to predict the last column, which is the % of silica in the iron ore concentrate.

**Objectives**

- Is it possible to predict % Silica Concentrate every minute?

- How many steps (hours) ahead can we predict % Silica in Concentrate? This would help engineers to act in predictive and optimized way, mitigatin the % of iron that could have gone to tailings.

- Is it possible to predict % Silica in Concentrate whitout using % Iron Concentrate column (as they are highly correlated)?

Source of data: **Kaggle**

**Project plan**

1. Data Exploration and Preprocessing 
     - Python: clustering and anomaly detection 
    
2. Correlation Analysis

3. Normalization

4. Variable 
    - Linear Regression
    - Random Forest
    
5. Partition of the data 

6. Evaluation metrics

7. Naive Benchmark 

8. Machine Learning 
    -selected methods: ,,.......
        
9. Sensitivity Analysis 

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv(r"C:\Users\marumom\Desktop\DATA ANALYSIS Projects 2023\MiningProcess_Flotation_Plant_Database.csv")
data

In [ ]:
data.columns

#for this study we can ignore the date, focus will be on the other columns but it's not necessary to delete the date column

In [ ]:
#missing values
data.isna().sum()
#no missing values

In [ ]:
data.info()
#later change the dtype to date and integer

In [ ]:
data.describe()

In [ ]:
data_sub= data[['% Iron Feed', '% Silica Feed', 'Starch Flow', 'Amina Flow',
       'Ore Pulp Flow', 'Ore Pulp pH', 'Ore Pulp Density',
       'Flotation Column 01 Air Flow', 'Flotation Column 02 Air Flow',
       'Flotation Column 03 Air Flow', 'Flotation Column 04 Air Flow',
       'Flotation Column 05 Air Flow', 'Flotation Column 06 Air Flow',
       'Flotation Column 07 Air Flow', 'Flotation Column 01 Level',
       'Flotation Column 02 Level', 'Flotation Column 03 Level',
       'Flotation Column 04 Level', 'Flotation Column 05 Level',
       'Flotation Column 06 Level', 'Flotation Column 07 Level',
       '% Iron Concentrate', '% Silica Concentrate']]
data_sub.head()

In [ ]:
data_sub = data_sub.replace(regex={',' : '.'}).astype(float)

#replace , with .
#data_sub[['% Iron Feed', '% Silica Feed', 'Starch Flow','Ore Pulp Density', '% Iron Concentrate', '% Silica Concentrate']] = data_sub[['% Iron Feed', '% Silica Feed', 'Starch Flow','Ore Pulp Density', '% Iron Concentrate', '% Silica Concentrate']].replace(regex={',' : '.'}).astype(float)
data_sub


In [ ]:
#check the shape of the dataframe

data_sub.shape

In [ ]:
#boxplots to look at the data for any outliers etc
#define a function called plot_boxplot
#df is the respective dataframe 
#var is the feature/variable

def plot_boxplot(df, var):
    df.boxplot(column =['% Iron Feed', '% Silica Feed', 'Starch Flow', 'Amina Flow',
       'Ore Pulp Flow', 'Ore Pulp pH', 'Ore Pulp Density',
       'Flotation Column 01 Air Flow', 'Flotation Column 02 Air Flow',
       'Flotation Column 03 Air Flow', 'Flotation Column 04 Air Flow',
       'Flotation Column 05 Air Flow', 'Flotation Column 06 Air Flow',
       'Flotation Column 07 Air Flow', 'Flotation Column 01 Level',
       'Flotation Column 02 Level', 'Flotation Column 03 Level',
       'Flotation Column 04 Level', 'Flotation Column 05 Level',
       'Flotation Column 06 Level', 'Flotation Column 07 Level',
       '% Iron Concentrate', '% Silica Concentrate'])
    plt.grid(False)
    plt.show()

In [ ]:
plot_boxplot(data_sub, [['% Iron Feed', '% Silica Feed', 'Starch Flow', 'Amina Flow',
       'Ore Pulp Flow', 'Ore Pulp pH', 'Ore Pulp Density',
       'Flotation Column 01 Air Flow', 'Flotation Column 02 Air Flow',
       'Flotation Column 03 Air Flow', 'Flotation Column 04 Air Flow',
       'Flotation Column 05 Air Flow', 'Flotation Column 06 Air Flow',
       'Flotation Column 07 Air Flow', 'Flotation Column 01 Level',
       'Flotation Column 02 Level', 'Flotation Column 03 Level',
       'Flotation Column 04 Level', 'Flotation Column 05 Level',
       'Flotation Column 06 Level', 'Flotation Column 07 Level',
       '% Iron Concentrate', '% Silica Concentrate']] )
#results are not readable so will create subplots

In [ ]:
data_sub.plot( kind = 'box', subplots=True, sharey=False, layout=(5,5), figsize=(10,6))
plt.tight_layout()
plt.show()

#improved view
#might explore better views at a later stage 

In [ ]:
#remove outliers using 2 step approach
#IQR

def outliers(df, var):
    Q1 = df[var].quantile(0.25)
    Q3= df[var].quantile(0.75)
    IQR = Q3 - Q1 
    
    lower = Q1 - 1.5*IQR
    upper = Q3 + 1.5*IQR
    
    list = data_sub.index[(df[var] < lower) | (df[var] > upper)]
    
    return list 


In [ ]:
#create an empty list to store the output indices from multiple columns 

index_list=[]

#extract outliers for all columns using for loop

for variable in ['% Iron Feed', '% Silica Feed', 'Starch Flow', 'Amina Flow',
       'Ore Pulp Flow', 'Ore Pulp pH', 'Ore Pulp Density',
       'Flotation Column 01 Air Flow', 'Flotation Column 02 Air Flow',
       'Flotation Column 03 Air Flow', 'Flotation Column 04 Air Flow',
       'Flotation Column 05 Air Flow', 'Flotation Column 06 Air Flow',
       'Flotation Column 07 Air Flow', 'Flotation Column 01 Level',
       'Flotation Column 02 Level', 'Flotation Column 03 Level',
       'Flotation Column 04 Level', 'Flotation Column 05 Level',
       'Flotation Column 06 Level', 'Flotation Column 07 Level',
       '% Iron Concentrate', '% Silica Concentrate']:
    index_list.extend(outliers(data_sub, variable))


In [ ]:
len(index_list)

In [ ]:
#define a function called remove which will result in a clean df 

def remove(df, list):
    list = sorted(set(list))
    df = df.drop(list)
    return df

In [ ]:
data_cleaned = remove(data_sub, index_list)

In [ ]:
data_cleaned.shape

In [ ]:
#plot boxplot again to check if there still are any outliers

data_cleaned.plot( kind = 'box', subplots=True, sharey=False, layout=(5,5), figsize=(10,6))
plt.tight_layout()

In [ ]:
#histogram to see data 
data_cleaned.hist()

In [ ]:
#checking for nan values 
data_cleaned.isna().sum()

In [ ]:
#reset index

data_cleaned.reset_index()

#drop the index column



In [ ]:
#plot the distribution of silica concentrate

import seaborn as sns

ax = sns.countplot(x='% Silica Concentrate', data=data_cleaned )

#return to fix the axes

**Correlation Analysis**

To examine if explanatory variables share the same linear relationship with the outcome variable in order to detect duplications of variables in the dataset.


In [ ]:
#correlation heat map

plt.figure(figsize=(16,16))
sns.heatmap(data_cleaned[['% Iron Feed', '% Silica Feed', 'Starch Flow', 'Amina Flow',
       'Ore Pulp Flow', 'Ore Pulp pH', 
       'Flotation Column 01 Air Flow', 'Flotation Column 02 Air Flow',
       'Flotation Column 03 Air Flow', 'Flotation Column 04 Air Flow',
       'Flotation Column 05 Air Flow', 'Flotation Column 06 Air Flow',
       'Flotation Column 07 Air Flow', 'Flotation Column 01 Level',
       'Flotation Column 02 Level', 'Flotation Column 03 Level',
       'Flotation Column 04 Level', 'Flotation Column 05 Level',
       'Flotation Column 06 Level', 'Flotation Column 07 Level',
       '% Iron Concentrate','% Silica Concentrate' ]].corr(), annot = True, fmt =".2f")
plt.show()

#we get the pearson correlatin coefficient from the heat map -r 
#a browse of the r coefficient shows us which variables are highly correlated and those that are not correlated.

**Normalization**

The variables are all numerical so no need to vectorize but they were stored on a different scale so each variable was normalized independently in order to put them on the same scale. 

Going forth, I did feature selection to reduce the number of input variables when developing a predictive model.
I based myself on the pearson correlation coeefficients that showed the correlations between variables. 
For now, I will set aside the 'Flotation Column 01 Air Flow', 'Flotation Column 02 Air Flow',
       'Flotation Column 03 Air Flow', 'Flotation Column 04 Air Flow',
       'Flotation Column 05 Air Flow', 'Flotation Column 06 Air Flow',
       'Flotation Column 07 Air Flow', 'Flotation Column 01 Level',
       'Flotation Column 02 Level', 'Flotation Column 03 Level',
       'Flotation Column 04 Level', 'Flotation Column 05 Level',
       'Flotation Column 06 Level', 'Flotation Column 07 Level' independent variables and explore the remaining variables. 

In [ ]:
from scipy.stats import zscore


In [ ]:
#calculate the zscores and drop zcores into new dataframe
data_norm = zscore(data_cleaned[['% Iron Feed', '% Silica Feed', 'Starch Flow', 'Amina Flow',
       'Ore Pulp Flow', 'Ore Pulp pH', 'Ore Pulp Density',
              '% Iron Concentrate','% Silica Concentrate' ]])

In [ ]:
data_norm.shape

In [ ]:
#visualize the trends of the remaining variables with the target variable % Silica Concentrate


In [ ]:
import plotly.express as px

fig = px.scatter_matrix(data_norm[['% Iron Feed', '% Silica Feed', 'Starch Flow', 'Amina Flow', 'Ore Pulp Flow', 'Ore Pulp pH',
       'Ore Pulp Density', '% Iron Concentrate', '% Silica Concentrate']])
fig.show()

In [ ]:
#take a closer look at %iron feed and %silica feed as well as %iron concentrate and %silica conc.

px.scatter(x=data_norm['% Iron Feed'], y =data_norm['% Silica Feed'])

#indicates a negative correlation, good spread of data points
#could indicate possible chemical reactions taking place iron ----> silica

In [ ]:
px.scatter(x=data_norm['% Iron Concentrate'], y =data_norm['% Silica Concentrate'])

In [ ]:
px.scatter(x=data_norm['% Iron Feed'], y =data_norm['% Iron Concentrate'])


In [ ]:
px.scatter(x=data_norm['% Silica Feed'], y =data_norm['% Silica Concentrate'])


In [ ]:
px.scatter(x=data_norm['% Iron Feed'], y =data_norm['% Silica Concentrate'])


**Training and testing models**

I will partition the Data into two parts for training and testing purpose: 70% of the entire dataset for training the selected models using 10-fold cross validation method and 30% for testing purpose. 

The respective training and validation dataset were randomly sampled to circumvent sampling bias


In [ ]:
#importing the required libraries
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.datasets import load_digits

digits = load_digits()

In [ ]:
#split data set into training and test dataset

from sklearn.model_selection import train_test_split

X = np.array(data_cleaned[['% Iron Feed', '% Silica Feed', 'Starch Flow', 'Amina Flow',
                  'Ore Pulp Flow', 'Ore Pulp pH', 'Ore Pulp Density','% Iron Concentrate']])

y = np.array(data_cleaned[['% Silica Concentrate']])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
#multiple linear regression 
lr = LinearRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(lr, X, y, cv=10)
scores

In [ ]:
#random forest
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

In [ ]:
scores = cross_val_score(rf, X, y, cv=10)
scores

In [ ]:
#import kfold library
from sklearn.model_selection import KFold
kf = KFold(n_splits = 10 )
kf.get_n_splits(X, y)